In [1]:

import os
import torch
import trimesh
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm as tqdm

In [2]:
def angle_between_vectors(v1, v2):
    return np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))


def extract_single_edge_features(mesh, edge_idx):
    edge = mesh.edges[edge_idx]
    adjacent_faces = mesh.face_adjacency_edges[edge_idx]

    # Compute dihedral angle
    dihedral_angle = mesh.face_adjacency_angles[edge_idx]  # Dihedral angle between the two adjacent faces

    # Compute inner angles (opposite angles in the two adjacent faces)
    inner_angles = []
    for face_idx in adjacent_faces:
        face = mesh.faces[face_idx]
        opposite_vertex = [v for v in face if v not in edge][0]
        opposite_angle = angle_between_vectors(mesh.vertices[opposite_vertex] - mesh.vertices[edge[0]],
                                               mesh.vertices[opposite_vertex] - mesh.vertices[edge[1]])
        inner_angles.append(opposite_angle)

    # Compute edge/height ratios for the two adjacent faces
    edge_lengths = mesh.edges_unique_length[edge_idx]  # Edge length
    heights = []
    for face_idx in adjacent_faces:
        face = mesh.faces[face_idx]
        opposite_vertex = [v for v in face if v not in edge][0]
        height = np.linalg.norm(np.cross(mesh.vertices[opposite_vertex] - mesh.vertices[edge[0]],
                                         mesh.vertices[opposite_vertex] - mesh.vertices[edge[1]])) / edge_lengths
        heights.append(height)

    # Collect the feature vector
    edge_feature = [dihedral_angle, inner_angles[0], inner_angles[1], edge_lengths / heights[0], edge_lengths / heights[1]]
    return edge_feature, edge[0], edge[1]

def extract_edge_features(mesh):
    edges = mesh.edges

    num_edges = len(edges)
    edge_features = []

    processed_edges = set()
    edge_idx = 0

    for edge_idx in range(num_edges):
        edge = edges[edge_idx]
        print(edge)
        v1 = edge[0]
        v2 = edge[1]
                
        print(f'Edge {edge_idx}: {v1} -> {v2}')
                
        adjacent_faces = mesh.face_adjacency_edges[v1]
        print(f'Adjacent faces v1: {mesh.face_adjacency_edges[v1]}')
        print(f'Adjacent faces v2: {mesh.face_adjacency_edges[v2]}')

        # Compute dihedral angle
        dihedral_angle = mesh.face_adjacency_angles[edge_idx]  # Dihedral angle between the two adjacent faces

        # Compute inner angles (opposite angles in the two adjacent faces)
        inner_angles = []
        for face_idx in adjacent_faces:
            face = mesh.faces[face_idx]
            opposite_vertex = [v for v in face if v not in edge][0]
            opposite_angle = angle_between_vectors(mesh.vertices[opposite_vertex] - mesh.vertices[edge[0]],
                                                mesh.vertices[opposite_vertex] - mesh.vertices[edge[1]])
            inner_angles.append(opposite_angle)

        # Compute edge/height ratios for the two adjacent faces
        edge_lengths = mesh.edges_unique_length[edge_idx]  # Edge length
        heights = []
        for face_idx in adjacent_faces:
            face = mesh.faces[face_idx]
            opposite_vertex = [v for v in face if v not in edge][0]
            height = np.linalg.norm(np.cross(mesh.vertices[opposite_vertex] - mesh.vertices[edge[0]],
                                            mesh.vertices[opposite_vertex] - mesh.vertices[edge[1]])) / edge_lengths
            heights.append(height)

        # Collect the feature vector
        edge_feature = [dihedral_angle, inner_angles[0], inner_angles[1], edge_lengths / heights[0], edge_lengths / heights[1]]
                
                
        edge_features.append(edge_feature)
        edge_idx += 1

    return np.array(edge_features)
  
  
def create_image_like_tensor(mesh):
    edge_features = extract_edge_features(mesh)
    num_edges = len(edge_features)
    num_features = len(edge_features[0])

    image_tensor = torch.zeros((1, num_edges, num_features))

    for edge_idx in range(num_edges):
        image_tensor[0, edge_idx, :] = torch.tensor(edge_features[edge_idx])

    return image_tensor


def save_image_tensor(image_tensor, output_path):
    torch.save(image_tensor, output_path)

def process_mesh_dataset(dataset_path, output_dir):
    """
    Process the mesh dataset and save the image-like tensors in the specified folder.

    Args:
        dataset_path (str): Path to the folder containing mesh .obj files.
        output_dir (str): Folder where the tensors will be saved.
    """
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".obj"):
                mesh_path = os.path.join(root, file)
                print(f"Processing {mesh_path} ... ")
               
                mesh = trimesh.load(mesh_path)
                image_tensor = create_image_like_tensor(mesh)

                # Create output path
                relative_path = os.path.relpath(mesh_path, dataset_path)
                tensor_output_path = os.path.join(output_dir, relative_path.replace('.obj', '.pt'))
                os.makedirs(os.path.dirname(tensor_output_path), exist_ok=True)

                save_image_tensor(image_tensor, tensor_output_path)
                print(f"Saved tensor for {file} at {tensor_output_path}")

# Example usage
dataset_path = "datasets/human/bouncing"  # Replace with your dataset folder containing .obj files
output_dir = "mesh_images"  # Folder where image-like tensors will be saved

process_mesh_dataset(dataset_path, output_dir)


Processing datasets/human/bouncing/mesh_0170.obj ... 
[   8 9892]
Edge 0: 8 -> 9892
Adjacent faces v1: [ 6 20]
Adjacent faces v2: [3409 3420]
[9892 9876]
Edge 1: 9892 -> 9876
Adjacent faces v1: [3409 3420]
Adjacent faces v2: [3325 3415]
[9876    8]
Edge 2: 9876 -> 8
Adjacent faces v1: [3325 3415]
Adjacent faces v2: [ 6 20]
[9876 9874]
Edge 3: 9876 -> 9874
Adjacent faces v1: [3325 3415]
Adjacent faces v2: [3385 3414]
[9874    8]
Edge 4: 9874 -> 8
Adjacent faces v1: [3385 3414]
Adjacent faces v2: [ 6 20]
[   8 9876]
Edge 5: 8 -> 9876
Adjacent faces v1: [ 6 20]
Adjacent faces v2: [3325 3415]
[9863   88]
Edge 6: 9863 -> 88
Adjacent faces v1: [3298 3411]
Adjacent faces v2: [75 76]
[88 87]
Edge 7: 88 -> 87
Adjacent faces v1: [75 76]
Adjacent faces v2: [25 76]
[  87 9863]
Edge 8: 87 -> 9863
Adjacent faces v1: [25 76]
Adjacent faces v2: [3298 3411]
[  88 9866]
Edge 9: 88 -> 9866
Adjacent faces v1: [75 76]
Adjacent faces v2: [3320 3412]
[9866    9]
Edge 10: 9866 -> 9
Adjacent faces v1: [3320 34

KeyboardInterrupt: 

In [ ]:
def mesh_to_pointcloud(mesh, num_points=1024, normalize=True):
    """Convert a trimesh mesh object to a point cloud."""
    point_cloud = mesh.sample(num_points)  # Randomly sample points from mesh surface
    if normalize:
        point_cloud -= np.mean(point_cloud, axis=0)
        point_cloud /= np.max(np.linalg.norm(point_cloud, axis=1))
    return point_cloud.astype(np.float32)


class MeshDataset(Dataset):
    def __init__(self, root_dir, num_points=1024, transform=None):
        self.root_dir = root_dir
        self.num_points = num_points
        self.transform = transform
        self.samples = []

        for class_name in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                obj_files = [f for f in os.listdir(class_dir) if f.endswith('.obj')]
                for obj_file in obj_files:
                    obj_path = os.path.join(class_dir, obj_file)
                    label = class_name 
                    self.samples.append((obj_path, label))

        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(sorted(set([s[1] for s in self.samples])))}
        self.idx_to_class = {idx: cls_name for cls_name, idx in self.class_to_idx.items()}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        obj_path, label = self.samples[idx]
        
        mesh = trimesh.load(obj_path)
        point_cloud = mesh_to_pointcloud(mesh, num_points=self.num_points, normalize=True)        
        
        label_idx = self.class_to_idx[label]

        if self.transform:
            point_cloud = self.transform(point_cloud)

        point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float32)
        label_tensor = torch.tensor(label_idx, dtype=torch.long)

        return point_cloud_tensor, label_tensor

root_dir = 'datasets/human'

dataset = MeshDataset(root_dir, num_points=1024)

dataset_size = len(dataset)
test_size = int(0.10 * dataset_size)
val_size = int(0.15 * dataset_size)
train_size = dataset_size - test_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


print(f'Number of samples: {len(dataset)}')
print(f'Classes: {dataset.class_to_idx}')
print(f'Number of classes: {len(dataset.class_to_idx)}')
print(dataset.samples)
for i, count in enumerate(np.bincount([dataset.class_to_idx[s[1]] for s in dataset.samples])):
    print(f'Class {i} - {dataset.idx_to_class[i]}: {count} samples')

print('Train size:', len(train_dataset))
print('Validation size:', len(val_dataset))
print('Test size:', len(test_dataset))

In [4]:
class SimplePointNet(nn.Module):
    def __init__(self, num_classes, input_dim=3):
        """
        Args:
            num_classes (int): Number of output classes.
            input_dim (int): Dimension of the input point cloud (default is 3 for x, y, z).
        """
        super(SimplePointNet, self).__init__()

        # Input transformation MLP
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 1024)

        # Batch normalization layers applied after the fully connected layers
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

        # Global feature vector aggregation
        self.maxpool = nn.MaxPool1d(1024)

        # Fully connected layers for classification
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 256)
        self.fc6 = nn.Linear(256, num_classes)

        # Batch normalization layers for the classification layers
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        # x: (batch_size, num_points, 3)
        batch_size = x.size(0)
        num_points = x.size(1)

        # Apply the first MLP layers
        x = F.relu(self.fc1(x))  # (batch_size, num_points, 64)
        x = self.bn1(x.transpose(1, 2)).transpose(1, 2)  # (batch_size, num_points, 64)
        
        x = F.relu(self.fc2(x))  # (batch_size, num_points, 128)
        x = self.bn2(x.transpose(1, 2)).transpose(1, 2)  # (batch_size, num_points, 128)
        
        x = F.relu(self.fc3(x))  # (batch_size, num_points, 1024)
        x = self.bn3(x.transpose(1, 2)).transpose(1, 2)  # (batch_size, num_points, 1024)

        # Global feature vector aggregation (max-pooling over points)
        x = x.permute(0, 2, 1)  # (batch_size, 1024, num_points)
        x = self.maxpool(x)  # (batch_size, 1024, 1)
        x = x.view(batch_size, -1)  # (batch_size, 1024)

        # Apply the fully connected layers for classification
        x = F.relu(self.bn4(self.fc4(x)))  # (batch_size, 512)
        x = F.relu(self.bn5(self.fc5(x)))  # (batch_size, 256)
        x = self.fc6(x)  # (batch_size, num_classes)

        return x


In [ ]:
num_classes = len(dataset.class_to_idx)

model = SimplePointNet(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for point_clouds, labels in train_loader:
        outputs = model(point_clouds)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

    # Validation
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for point_clouds, labels in val_loader:
            outputs = model(point_clouds)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')
    
    torch.save(model.state_dict(), f'checkpoints/pointnet_epoch_{epoch+1}.pth')

torch.save(model.state_dict(), 'checkpoints/pointnet.pth')

In [ ]:
test_model = SimplePointNet(num_classes=num_classes)  
test_model.load_state_dict(torch.load('checkpoints/pointnet.pth'))
test_model.eval()

correct = 0
total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for point_clouds, labels in test_loader:
        outputs = model(point_clouds)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())
        
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=dataset.idx_to_class.values(), yticklabels=dataset.idx_to_class.values())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()